# Digital Elevation Model Download

This notebook downloads the Digital Elevation Model - DEM, from The National Map using the TNM Rest API. API documentation is [available here](https://tnmaccess.nationalmap.gov/api/v1/docs).

In [1]:
# Import modules
import requests
from pathlib import Path
import json
import sys
import os
import warnings
import re
from datetime import datetime
from IPython.display import display, Markdown
import json
from dotenv import load_dotenv


In [2]:
# Root path
project_root_path = Path.cwd().parent.parent

# Supress warnings
warnings.filterwarnings("ignore")

In [3]:
# Add 'src' to system path
sys.path.append(str(project_root_path / 'src'))

# Import modules
from data_download.download import download_GeoTIFF

In [4]:
# Load tokens
load_dotenv(project_root_path / '.env')

NOAA_API_TOKEN = os.getenv('NOAA_API_TOKEN')

## Screening and Availability Assessment of 1/3 Arc-Second Resolution DEM

This section focuses on accessing and evaluating the availability of high-resolution (1/3 arc-second, approximately 10-meter resolution) Digital Elevation Model (DEM) data for the defined study area using the U.S. Geological Survey (USGS) National Map (TNM) API.

The accompanying code queries the TNM API to retrieve metadata for all 1/3 arc-second DEM tiles that intersect the spatial extent of the study region. This metadata includes essential information for each available tile, such as unique identifiers, spatial extent, and direct download links.

Following the retrieval of the list of available high-resolution DEM tiles within the study area, the total cumulative file size required to download all these tiles is calculated. This calculation is performed to assess the overall data volume and evaluate the computational resources (storage, memory, processing time) needed to effectively work with the 1/3 arc-second resolution data. Based on this total size assessment, a determination will be made regarding the feasibility of utilizing the high-resolution DEMs or if transitioning to a coarser resolution dataset is necessary to ensure efficient processing throughout the project.

In [6]:
# Load the study area bounding box
study_area_bbox_path = project_root_path / 'data' / 'silver'/ 'geo' /'json' / 'study_area_bbox.json'
with open(study_area_bbox_path, 'r') as f:
    study_area_bbox_dic = json.load(f)

# Build the box as a string for feeding the request in the parameters
corners = ['bottom_left', 'bottom_right', 'top_right', 'top_left']
pairs = [f"{study_area_bbox_dic[corner][0]} {study_area_bbox_dic[corner][1]}" for corner in corners]
bbox = ",".join(pairs)

# Define the base URL for the TNM API
base_url = "https://tnmaccess.nationalmap.gov/api/v1/"

product = "National Elevation Dataset (NED) 1/3 arc-second"

# Define parameters for the API request to query available datasets
params = {
    "polygon": bbox,  # Specify the area to search for
    "datasets": product,  
    "outputFormat": "JSON",
    "max": 1000
}

# Send a GET request to the API
response_dem_availabilty = requests.get(base_url + "products", params=params)

# Check if the request was successful
if response_dem_availabilty.status_code == 200:
    # Parse the JSON response
    response_dem_availabilty_json = response_dem_availabilty.json()
    
    # Display the dataset information
    display(Markdown(f'**{len(response_dem_availabilty_json.get("items",[]))} files were found. Displaying the first 5:**\n '))
    count =  0
    for dataset in response_dem_availabilty_json.get("items", []):
        count += 1
        if count > 5:
            break
        print(f"- Name: {dataset['title']}")
        print(f"  Publication Date: {dataset['publicationDate']}")
        print(f"  Description: {dataset['body']}")
        print(f"  Metadata URL: {dataset['metaUrl']}\n")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response_dem_availabilty.status_code}")

**379 files were found. Displaying the first 5:**
 

- Name: USGS 1/3 Arc Second n36w078 20250507
  Publication Date: 2025-05-07
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These

From the response returned, there area **379** GeoTiff file to download. However, there are multiple file for the same area, from different dates (for example, file name *USGS 1/3 Arc Second n36w081 20240510* and *USGS 1/3 Arc Second n36w081 20240611* are both for area n36w081, but for dates *2024-05-10* and *2024-06-11* respectvely). For this study it will always be used the latest file. 

The code below, filters the latest file for each region. Before, proceeding, however, the raw list of available files is saved in the bronze data folder. |

In [32]:
# Save raw list
available_dem_list_path = project_root_path / 'data/bronze/json_docs/available_10m_dem_list.json'
available_dem_list_path.parent.mkdir(parents=True, exist_ok=True)
with open(available_dem_list_path, 'w', encoding='utf-8') as f:
    json.dump(response_dem_availabilty_json, f, ensure_ascii=False, indent=4)

In [10]:
def filter_latest_geotiff(data):

    region_latest = {}
    
    for item in data:
        # Extract region from title using regex pattern (e.g., n41w074)
        match = re.search(r'n\d+w\d+', item['title'], re.IGNORECASE)
        if not match:
            continue
        region = match.group(0).lower()

        # Parse publication date
        pub_date = datetime.strptime(item['publicationDate'], '%Y-%m-%d')

        # If region is new or found a later publication date, update the entry
        if region not in region_latest or pub_date > datetime.strptime(region_latest[region]['publicationDate'], '%Y-%m-%d'):
            region_latest[region] = item

    # Return unique latest items for each region
    return list(region_latest.values())

dem_all_items = response_dem_availabilty_json.get("items",[])
dem_filtered_data = filter_latest_geotiff(data = dem_all_items)

# Display the dataset information
display(Markdown(f'**{len(dem_filtered_data)} unique files were found. Displaying the first 5:**\n '))
count =  0
for dataset in dem_filtered_data:
    count += 1
    if count > 5:
        break
    print(f"- Name: {dataset['title']}")
    print(f"  Publication Date: {dataset['publicationDate']}")
    print(f"  Description: {dataset['body']}")
    print(f"  Metadata URL: {dataset['metaUrl']}\n")


**104 unique files were found. Displaying the first 5:**
 

- Name: USGS 1/3 Arc Second n36w078 20250507
  Publication Date: 2025-05-07
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These

In [33]:
dem_filtered_data[0]

{'title': 'USGS 1/3 Arc Second n36w078 20250507',
 'moreInfo': 'This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as [...]',
 'sourceId': '681c128ed4be0260c2c465c7',
 'sourceName': 'ScienceBase',
 'sourceOriginId': None,
 'sourceOriginName': 'gda',
 'metaUrl': 'https://www.sciencebase.gov/catalog/item/681c128e

**Get the total size**

In [28]:
try:
    # Use a HEAD request to get only the headers
    url = dem_filtered_data[0]['downloadURL']
    response = requests.head(url)

    # Check if the request was successful
    response.raise_for_status()

    # Get the Content-Length header, which is the file size in bytes
    content_length = response.headers.get('Content-Length')

    if content_length:
        file_size_bytes = int(content_length)
        print(f"File: {url}")
        print(f"  Size: {file_size_bytes/1e6} mega bytes")
        print("-" * 20)
    else:
        print(f"Could not get Content-Length for {url}")
        print("-" * 20)

except requests.exceptions.RequestException as e:
    print(f"Error fetching metadata for {url}: {e}")
    print("-" * 20)

File: https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w078/USGS_13_n36w078_20250507.tif
  Size: 517.373038 mega bytes
--------------------


In [31]:
def get_total_tiff_size(file_urls):
    total_size_bytes = 0
    processed_count = 0 # To keep track of successfully processed files

    # Iterate through the URLs and get file sizes
    for url in file_urls:
        try:
            # Use a HEAD request to get only the headers
            # Set a reasonable timeout to avoid hanging indefinitely
            response = requests.head(url, timeout=10)

            # Check if the request was successful (status code 2xx)
            response.raise_for_status()

            # Get the Content-Length header, which is the file size in bytes
            content_length = response.headers.get('Content-Length')

            if content_length:
                try:
                    file_size_bytes = int(content_length)
                    total_size_bytes += file_size_bytes
                    processed_count += 1
                    print(f"Successfully retrieved size for {url}: {file_size_bytes} bytes")
                except ValueError:
                    print(f"Warning: Could not convert Content-Length to integer for {url}")
            else:
                print(f"Warning: Could not get Content-Length for {url}")

        except requests.exceptions.Timeout:
            print(f"Error: Request timed out for {url}")
        except requests.exceptions.ConnectionError:
             print(f"Error: Failed to connect to {url}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching metadata for {url}: {e}")

    print(f"\n--- Summary ---")
    print(f"Attempted to process {len(file_urls)} URLs.")
    print(f"Successfully processed {processed_count} URLs.")
    print(f"Total size of processed files: {total_size_bytes/1e6} Mega bytes")
    print(f"---------------\n")

    return total_size_bytes/1e6

# Get list of download links
download_urls_list = [item['downloadURL'] for item in dem_filtered_data]

# Call the function and get the total size
overall_total_size = get_total_tiff_size(download_urls_list)

Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w078/USGS_13_n36w078_20250507.tif: 517373038 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w079/USGS_13_n36w079_20250507.tif: 497376568 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w080/USGS_13_n36w080_20240611.tif: 494107267 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w081/USGS_13_n36w081_20240611.tif: 479546028 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w082/USGS_13_n36w082_20240611.tif: 482156693 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n36w083/USGS_13_n36w083_20220512.tif: 481732380 bytes
Successfully retrieved

The total size of GeoTiff files is aproximately **46 Gb**. Which is beyond the scope of this study. Next, the total the 1 arc (30m) resolution is assessed. 

## Screening and Availability Assessment of 1 Arc-Second Resolution DEM

In [11]:
product = "National Elevation Dataset (NED) 1 arc-second"

# Define parameters for the API request to query available datasets
params = {
    "polygon": bbox,  # Specify the area to search for
    "datasets": product,  
    "outputFormat": "JSON",
    "max": 1000
}

# Send a GET request to the API
response_dem_1arc_availabilty = requests.get(base_url + "products", params=params)

# Check if the request was successful
if response_dem_1arc_availabilty.status_code == 200:
    # Parse the JSON response
    response_dem_1arc_availabilty_json = response_dem_1arc_availabilty.json()
    
    # Display the dataset information
    display(Markdown(f'**{len(response_dem_1arc_availabilty_json.get("items",[]))} files were found. Displaying the first 5:**\n '))
    count =  0
    for dataset in response_dem_1arc_availabilty_json.get("items", []):
        count += 1
        if count > 5:
            break
        print(f"- Name: {dataset['title']}")
        print(f"  Publication Date: {dataset['publicationDate']}")
        print(f"  Description: {dataset['body']}")
        print(f"  Metadata URL: {dataset['metaUrl']}\n")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response_dem_1arc_availabilty.status_code}")

**383 files were found. Displaying the first 5:**
 

- Name: USGS 1 Arc Second n36w078 20250507
  Publication Date: 2025-05-07
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These dat

In [ ]:
# Save raw list
available_30m_dem_list_path = project_root_path / 'data/bronze/json_docs/available_30m_dem_list.json'
available_30m_dem_list_path.parent.mkdir(parents=True, exist_ok=True)
with open(available_30m_dem_list_path, 'w', encoding='utf-8') as f:
    json.dump(response_dem_1arc_availabilty_json, f, ensure_ascii=False, indent=4)

**Filter latest 1 Arc-Second  DEM files**

In [12]:
dem_1arc_all_items = response_dem_1arc_availabilty_json.get("items",[])
dem_1arc_filtered_data = filter_latest_geotiff(data = dem_1arc_all_items)

# Display the dataset information
display(Markdown(f'**{len(dem_filtered_data)} unique 1 arc-second files were found. Displaying the first 5:**\n '))
count =  0
for dataset in dem_filtered_data:
    count += 1
    if count > 5:
        break
    print(f"- Name: {dataset['title']}")
    print(f"  Publication Date: {dataset['publicationDate']}")
    print(f"  Description: {dataset['body']}")
    print(f"  Metadata URL: {dataset['metaUrl']}\n")

**104 unique 1 arc-second files were found. Displaying the first 5:**
 

- Name: USGS 1/3 Arc Second n36w078 20250507
  Publication Date: 2025-05-07
  Description: This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as new data become available. Seamless 3DEP data are derived from diverse source data that are processed to a common coordinate system and unit of vertical measure. These

**Get the total size of 1 arc-second resolution DEM files**

In [38]:
# Get list of download links
download_1arc_urls_list = [item['downloadURL'] for item in dem_1arc_filtered_data]

# Call the function and get the total size
overall_total_size = get_total_tiff_size(download_1arc_urls_list)

Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w078/USGS_1_n36w078_20250507.tif: 61082287 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w079/USGS_1_n36w079_20250507.tif: 59561408 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w080/USGS_1_n36w080_20240611.tif: 59565558 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w081/USGS_1_n36w081_20240611.tif: 57343520 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w082/USGS_1_n36w082_20240611.tif: 57611101 bytes
Successfully retrieved size for https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n36w083/USGS_1_n36w083_20220512.tif: 57595140 bytes
Successfully retrieved size for https://

In [39]:
# Save raw list
filtered_30m_dem_list_path = project_root_path / 'data/silver/json_docs/filtered_30m_dem_list.json'
filtered_30m_dem_list_path.parent.mkdir(parents=True, exist_ok=True)
with open(filtered_30m_dem_list_path, 'w', encoding='utf-8') as f:
    json.dump(dem_1arc_filtered_data, f, ensure_ascii=False, indent=4)

The total size of 1 Arc-Second GeoTiff files is about **5.6 Gb**, significantly smaller than the 1/3 Arc-Second resolution. Although a coarser resolution does not offer fine details as higher resolution, the overall size, of 1/3 Arc-Second does not justify the processing capacity that should be added to this project. Therefore, 1 Arc-Second resolution is chosen for this study. 

## 1 Arc-Second DEM Download

This section downloads the 1 Arc-Second geoTiff files for the study area.

In [40]:
# Load document containing download link
filtered_30m_dem_list_path = project_root_path / 'data/silver/json_docs/filtered_30m_dem_list.json'
with open(filtered_30m_dem_list_path, 'r') as file:
    dem_list = json.load(file)

count = 0
total_files = len(dem_list)

# Download the DEM 1 arcsec file
for item in dem_list:
    count += 1
    try:
        url = item.get('downloadURL')
        
        item_base_name = os.path.basename(url)
        item_local_path = project_root_path / 'data/bronze' / 'geo' / 'raster' / 'dem1arcsec' / item_base_name
        
        if not item_local_path.exists():
            print(f'Downloading {item.get("title")} file {count} of {total_files}...')
            download_GeoTIFF(url=url, filename=item_local_path, chunk_size=1024*1024)
            
        else:
            print(f'{item_base_name} already exists.\n')

    except Exception as err:
        print(f'Failed to download file: {err}')
        continue

Downloaded: USGS_1_n36w078_20250507.tif.00%)
Function 'download_GeoTIFF' executed in 11.0089 seconds.
Downloaded: USGS_1_n36w079_20250507.tif.00%)
Function 'download_GeoTIFF' executed in 12.6428 seconds.
Downloaded: USGS_1_n36w080_20240611.tif.00%)
Function 'download_GeoTIFF' executed in 11.3432 seconds.
Downloaded: USGS_1_n36w081_20240611.tif.00%)
Function 'download_GeoTIFF' executed in 13.0830 seconds.
Downloaded: USGS_1_n36w082_20240611.tif.00%)
Function 'download_GeoTIFF' executed in 9.8107 seconds.
Downloaded: USGS_1_n36w083_20220512.tif.00%)
Function 'download_GeoTIFF' executed in 11.5658 seconds.
Downloaded: USGS_1_n36w084_20220725.tif.00%)
Function 'download_GeoTIFF' executed in 9.9028 seconds.
Downloaded: USGS_1_n36w085_20220725.tif.00%)
Function 'download_GeoTIFF' executed in 9.5863 seconds.
Downloaded: USGS_1_n36w086_20220729.tif.00%)
Function 'download_GeoTIFF' executed in 10.1834 seconds.
Downloaded: USGS_1_n36w087_20230130.tif.00%)
Function 'download_GeoTIFF' executed in 